In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [19]:
data_dir = "HERRERA_NATIVI_VLADIMIR"
metadata_Vlad = pd.read_csv("HERRERA_NATIVI_VLADIMIR/metadata.csv")
metadata_Javi = pd.read_csv("PENA_CASTANO_JAVIER/metadata.csv")

# La prof ma parlé d'un tensorboard

In [20]:
print(f"Vladi : {metadata_Vlad.shape} | Javi : {metadata_Javi.shape}")

Vladi : (5627, 13) | Javi : (5477, 12)


In [21]:
# Fusionner les datasets
#metadata_combined = pd.concat([metadata_Vlad, metadata_Javi])

# Supprimer les doublons en gardant la première occurrence
#metadata_combined = metadata_combined.drop_duplicates(subset="Image Index", keep="first")

# Afficher un aperçu
#print(metadata_combined.head())

# Sauvegarder le dataset fusionné si besoin
#metadata_combined.to_csv("metadata_combined.csv", index=False)

#print(f"Merged  : {metadata_combined.shape}")

In [22]:
# Elimination des colonnes non necessaire pour notre étude initial
colonnes_a_eliminer = ['OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]']
dataset_col_utiles = metadata_Vlad.drop(columns=colonnes_a_eliminer, inplace=False) 
dataset_col_utiles

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train
0,00000006_000.png,No Finding,0,6,81,M,PA,1,True
1,00000025_000.png,Effusion,0,25,71,M,PA,1,True
2,00000029_000.png,No Finding,0,29,59,F,PA,1,False
3,00000072_000.png,Atelectasis,0,72,67,F,PA,1,True
4,00000090_000.png,No Finding,0,90,67,F,PA,1,True
...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,No Finding,0,30752,64,F,AP,1,True
5623,00030772_000.png,No Finding,0,30772,26,F,AP,1,True
5624,00030772_001.png,Consolidation,1,30772,26,F,AP,1,True
5625,00030772_002.png,Consolidation,2,30772,26,F,AP,1,True


In [23]:
# Encodage des colonnes a valeurs non continue 

# Encodage binaire pour Patient Gender et View Position
dataset_col_utiles['Patient Gender'] = (dataset_col_utiles['Patient Gender'] == 'M').astype(int) # M est la réference et donc sera associé a la valeur 0 
dataset_col_utiles['View Position'] = (dataset_col_utiles['View Position'] == 'PA').astype(int) # PA est la réference et donc sera associé a la valeur 0  

# On effectue un encodage binaire pour finding labels du fait qu'on s'interesse uniquement au fait d'être malade ou pas (on n'essaye pas de prédire les maladies)
dataset_col_utiles['Finding Labels'] = (dataset_col_utiles['Finding Labels'] == 'No Finding').astype(int) # No Finding est la réference et donc sera associé a la valeur 0 (tous le reste des maladies seront mise a 1)

""" 
# Encodage Several-Hot pour Finding Labels
mlb = MultiLabelBinarizer()
df_maladies = pd.DataFrame(mlb.fit_transform(dataset_col_utiles['Finding Labels'].str.split('|')), 
                           columns=mlb.classes_)

# Concaténer avec le dataset de notre étude
dataset_encod = pd.concat([dataset_col_utiles, df_maladies], axis=1)

# Supprimer la colonne Finding Labels originale
dataset_encod.drop(columns=['Finding Labels'], inplace=True)

"""

# dataset_encod.head()

" \n# Encodage Several-Hot pour Finding Labels\nmlb = MultiLabelBinarizer()\ndf_maladies = pd.DataFrame(mlb.fit_transform(dataset_col_utiles['Finding Labels'].str.split('|')), \n                           columns=mlb.classes_)\n\n# Concaténer avec le dataset de notre étude\ndataset_encod = pd.concat([dataset_col_utiles, df_maladies], axis=1)\n\n# Supprimer la colonne Finding Labels originale\ndataset_encod.drop(columns=['Finding Labels'], inplace=True)\n\n"

In [24]:
dataset_col_utiles

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train
0,00000006_000.png,1,0,6,81,1,1,1,True
1,00000025_000.png,0,0,25,71,1,1,1,True
2,00000029_000.png,1,0,29,59,0,1,1,False
3,00000072_000.png,0,0,72,67,0,1,1,True
4,00000090_000.png,1,0,90,67,0,1,1,True
...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,1,0,30752,64,0,0,1,True
5623,00030772_000.png,1,0,30772,26,0,0,1,True
5624,00030772_001.png,0,1,30772,26,0,0,1,True
5625,00030772_002.png,0,2,30772,26,0,0,1,True


In [25]:
dataset_col_utiles

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train
0,00000006_000.png,1,0,6,81,1,1,1,True
1,00000025_000.png,0,0,25,71,1,1,1,True
2,00000029_000.png,1,0,29,59,0,1,1,False
3,00000072_000.png,0,0,72,67,0,1,1,True
4,00000090_000.png,1,0,90,67,0,1,1,True
...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,1,0,30752,64,0,0,1,True
5623,00030772_000.png,1,0,30772,26,0,0,1,True
5624,00030772_001.png,0,1,30772,26,0,0,1,True
5625,00030772_002.png,0,2,30772,26,0,0,1,True


In [26]:

# Sauvegarder une fois que le pretraitement sera fait 
dataset_col_utiles.to_csv(f"{data_dir}/metadata_encod.csv", index=False) # sauvegarde de csv encodé 

In [27]:
dataset_col_utiles = dataset_col_utiles.loc[dataset_col_utiles['Patient Age'] <= 120] # On élimine les patients qui ont plus de 120 ans 

